# Ballpark Factor

Load the necessary packages

In [1]:
import requests
import lxml.html as lh
import pandas as pd

This is a function we'll use later to try turning data elements into integers, floats, or datetimes.

In [2]:
from datetime import datetime

def nice_data(d):
  #If we can turn it into an integer, then do that
  try:
    d=int(d)
  except:
  #If it's not an integer, it might be a float
    try:
      d = float(d)
    except:
      #If it's not a float, it might be a date
      try:
        d = datetime.strptime(d, '%m/%d/%Y')
      except:
        pass
  return(d)


the team names and numbers

In [3]:
teams =["BC", "GOC", "GRC", "HU", "INWU", "MAR", "MVNU", "SAU", "SFIN", "TAYL"]
t_nums = [1629, 1678, 1679, 1688, 1694, 1717, 1736, 1780, 1805, 1784]

Scrape the webpages

In [4]:
urls = ['http://www.dakstats.com/WebSync/Pages/Team/TeamSchedule.aspx?association=10&sg=MBA&sea=NAIMBA_2019&team=' +
        str(num) for num in t_nums]
#Create a handle, page, to handle the contents of the website
pages = [requests.get(url) for url in urls]
#Store the contents of the website under doc
docs = [lh.fromstring(page.content) for page in pages]
#Parse data that are stored between <table>..</table> of HTML
team_tables = [doc.xpath('//table') for doc in docs]

The for-loop below helps us find which tables on the website we need.  It appears that the header is the 33rd table, and the actual data is in the 35th table.

In [5]:
for i in range(len(team_tables[2])):
  print(i, team_tables[2][i].xpath('.//tr/td//text()'))

0 ['\r\n                    ', '\r\n                ']
1 ['\r\n            ', '\r\n                ', '\r\n                    ', '\r\n                        ', '\r\n                    ', '\r\n                    ', '\r\n                    ', '\r\n\t\t\t\t    ', '\r\n                        ', '\r\n\t', '\r\n\t\t', '\r\n                                    ', 'Browse by School', '\r\n                                ', '\r\n\t\t', '\r\n                                    ', 'Member Login', '\r\n                                ', '\r\n\t\t', '\r\n                                    ', "FAQ's", '\r\n                                ', '\r\n\t', '\r\n', '\r\n\r\n                    ', '\r\n                ', '\r\n                ', '\r\n\t\t\t\t    ', '\r\n                        ', '\r\n                            ', '\r\n                                ', '\r\n                                    ', 'Scoreboards', '\r\n                                ', '\r\n                             

In [46]:
headers = [tables[33].xpath('.//tr/td//text()') for tables in team_tables]
print(headers[2])

['Date', 'Opponent', 'H/A', 'Time', 'W/L']


In [47]:
team_elements = [tables[35].xpath('.//tr/td//text()') for tables in team_tables]
print(team_elements[2])

['\r\n    ', '2/27/2019', ' Lourdes (Ohio)', 'N', ' 3-4 ', 'L', '\r\n          ', '\r\n    ', '\r\n    ', '\r\n    ', '\r\n', '\r\n        ', '\r\n    ', '2/27/2019', ' Lourdes (Ohio)', 'N', ' 4-8 ', 'L', '\r\n          ', '\r\n    ', '\r\n    ', '\r\n    ', '\r\n', '\r\n        ', '\r\n    ', '3/2/2019', ' Cornerstone (Mich.)', 'N', ' 3-4 ', 'L', '\r\n          ', '\r\n    ', '\r\n    ', '\r\n    ', '\r\n', '\r\n        ', '\r\n    ', '3/2/2019', 'Trinity Baptist', 'N', ' 5-1 ', 'W', '\r\n          ', '\r\n    ', '\r\n    ', '\r\n    ', '\r\n', '\r\n        ', '\r\n    ', '3/4/2019', ' Michigan-Dearborn', 'N', ' 13-1 ', 'W', '\r\n          ', '\r\n    ', '\r\n    ', '\r\n    ', '\r\n', '\r\n        ', '\r\n    ', '3/5/2019', ' Rochester (Mich.)', 'N', ' 24-4 ', 'W', '\r\n          ', '\r\n    ', '\r\n    ', '\r\n    ', '\r\n', '\r\n        ', '\r\n    ', '3/6/2019', ' Robert Morris (Ill.)', 'N', ' 10-9 ', 'W', '\r\n          ', '\r\n    ', '\r\n    ', '\r\n    ', '\r\n', '\r\n        

In [48]:
team_good_els = [[nice_data(i) for i in elements if "\r" not in i] for elements in team_elements]
rows = [[good_els[i:i + 5] for i in range(0, len(good_els), 5)] for good_els in team_good_els]
rows[2][:2]

[[datetime.datetime(2019, 2, 27, 0, 0), ' Lourdes (Ohio)', 'N', ' 3-4 ', 'L'],
 [datetime.datetime(2019, 2, 27, 0, 0), ' Lourdes (Ohio)', 'N', ' 4-8 ', 'L']]

In [52]:
dfc = [pd.DataFrame(columns = headers[i], data = rows[i]) for i in range(len(headers))]
dfc[3][:5]

,Date,Opponent,H/A,Time,W/L
0,2019-02-08,Milligan (Tenn.),A,7-5,W
1,2019-02-08,Milligan (Tenn.),A,7-4,W
2,2019-02-09,Concordia (Mich.),N,2-0,W
3,2019-02-22,IU Southeast (Ind.),N,0-2,L
4,2019-02-22,IU Southeast (Ind.),N,7-13,L


Get only the conference games

In [54]:
conf_df = [df[df.Opponent.str.contains("*", regex = False)] for df in dfc]
conf_df[3][:5]

,Date,Opponent,H/A,Time,W/L
9,2019-03-08,Taylor (Ind.) *,A,7-6,W
10,2019-03-11,Taylor (Ind.) *,A,2-0,W
11,2019-03-11,Taylor (Ind.) *,A,1-5,L
12,2019-03-18,Spring Arbor (Mich.) *,H,6-5,W
13,2019-03-19,Spring Arbor (Mich.) *,H,8-0,W
